# BNN-BN
This notebook demonstrates the implementation of the BNN-BN Paper
# Steps to binarize the model
- Load Dataset and DataLoader
- Create the teacher model to be binarized
- Load the configuration YAML file
- create `BNNBN object` and pass the student model, dataloader, configuration
- Binarize the model by using `compress_model` method

In [1]:
import sys
import os
import shutil
sys.path.append("../../")
sys.path.append("../../../../../")

import torch
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
from trailmet.algorithms.binarize.utils import *
from trailmet.algorithms.binarize.BNNBN import BNNBN
from trailmet.models.models_bnnbn.Qa_reactnet_18_bn import birealnet18 as Qa_reactnet_18_bn
from trailmet.models.models_bnnbn.Qa_reactnet_18_none import birealnet18 as Qa_reactnet_18_none
from trailmet.models.models_bnnbn.Qa_reactnet_18_bf import birealnet18 as Qa_reactnet_18_bf

from trailmet.models.models_bnnbn.Qaw_reactnet_18_bn import birealnet18 as Qaw_reactnet_18_bn
from trailmet.models.models_bnnbn.Qaw_reactnet_18_none import birealnet18 as Qaw_reactnet_18_none
from trailmet.models.models_bnnbn.Qaw_reactnet_18_bf import birealnet18 as Qaw_reactnet_18_bf

#reactnet-A
from trailmet.models.models_bnnbn.Qa_reactnet_A_bn import reactnet as Qa_reactnet_A_bn
from trailmet.models.models_bnnbn.Qa_reactnet_A_none import reactnet as Qa_reactnet_A_none
from trailmet.models.models_bnnbn.Qa_reactnet_A_bf import reactnet as Qa_reactnet_A_bf

from trailmet.models.models_bnnbn.Qaw_reactnet_A_bn import reactnet as Qaw_reactnet_A_bn
from trailmet.models.models_bnnbn.Qaw_reactnet_A_none import reactnet as Qaw_reactnet_A_none
from trailmet.models.models_bnnbn.Qaw_reactnet_A_bf import reactnet as Qaw_reactnet_A_bf

import torchvision

import yaml

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]='1' 

In [3]:
root_dir = './'
import os
with open(os.path.join(root_dir,"bnnbn_cifar100.yaml"),'r') as stream:
    kwargs = yaml.safe_load(stream)
kwargs

{'DATASET': 'c100',
 'arch': 'reactnet-A',
 'binary_w': True,
 'bn_type': 'none',
 'batch_size': 256,
 'workers': 4,
 'valid_size': 0.1,
 'num_train': 0,
 'num_classes': 100,
 'insize': 32,
 'loss_type': 'ce',
 'teacher': 'resnet34',
 'teacher_weight': '',
 'label_smooth': 0.1,
 'pretrained': '',
 'resume': False,
 'save': './saved_weights',
 'epoch': 5,
 'agc': True,
 'clip_value': 0.04,
 'weight_decay': 0,
 'learning_rate': 0.001}

In [4]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(1024)  

> SEEDING DONE


# Defining the Model 

In [5]:
if kwargs['arch'] == 'reactnet-18':
    print('* Model = ReActNet-18')
    if kwargs['binary_w']:
        print('* Binarize both activation and weights')
        if kwargs['bn_type'] == 'bn':
            print('* with BN')
            model = Qaw_reactnet_18_bn(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'none':
            print('* without BN')
            model = Qaw_reactnet_18_none(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'bf':
            print('* BN-Free')
            model = Qaw_reactnet_18_bf(num_classes=kwargs['num_classes'])

    else:
        print('* Binarize only activation')
        if kwargs['bn_type'] == 'bn':
            print('* with BN')
            model = Qa_reactnet_18_bn(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'none':
            print('* without BN')
            model = Qa_reactnet_18_none(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'bf':
            print('* BN-Free')
            model = Qa_reactnet_18_bf(num_classes=kwargs['num_classes'])


elif kwargs['arch'] == 'reactnet-A':
    print('* Model = reactnet-A')
    if kwargs['binary_w']:
        print('* Binarize both activation and weights')
        if kwargs['bn_type'] == 'bn':
            print('* with BN')
            model = Qaw_reactnet_A_bn(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'none':
            print('* without BN')
            model = Qaw_reactnet_A_none(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'bf':
            print('* BN-Free')
            model = Qaw_reactnet_A_bf(num_classes=kwargs['num_classes'])

    else:
        print('* Binarize only activation')
        if kwargs['bn_type'] == 'bn':
            print('* with BN')
            model = Qa_reactnet_A_bn(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'none':
            print('* without BN')
            model = Qa_reactnet_A_none(num_classes=kwargs['num_classes'])
        elif kwargs['bn_type'] == 'bf':
            print('* BN-Free')
            model = Qa_reactnet_A_bf(num_classes=kwargs['num_classes'])

* Model = reactnet-A
* Binarize both activation and weights
* without BN


# Augmentation, Dataset and DataLoaders

In [6]:
# Augmentation
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(crop_scale, 1.0)),
    Lighting(lighting_param),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])

valid_transform = transforms.Compose([
        transforms.Resize(36),
        transforms.CenterCrop(32),
        transforms.ToTensor()
    ])
test_transform = valid_transform

input_transforms = {
    'train': train_transform, 
    'val': valid_transform, 
    'test': test_transform}

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

# Create Dataset
cifar100_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = root_dir,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.15,
                                        transform = input_transforms,
                                        target_transform = target_transforms
                                        )
# Define DataLoaders
train_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['train'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['train_sampler'],
        num_workers=kwargs['workers']
    )
val_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['val'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['val_sampler'],
        num_workers=kwargs['workers']
    )
test_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['test'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['test_sampler'],
        num_workers=kwargs['workers']
    )

dataloaders = {
        'train': train_loader100, 'val': val_loader100, "test": test_loader100
}


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# ReActNet Object

In [7]:
a = BNNBN(model, dataloaders, **kwargs)

# Binarizing the model

In [8]:
fin = a.compress_model()

reactnet(
  (feature): ModuleList(
    (0): firstconv3x3(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    )
    (1): BasicBlock(
      (move11): LearnableBias()
      (binary_3x3): HardBinaryConv()
      (move12): LearnableBias()
      (prelu1): PReLU(num_parameters=32)
      (move13): LearnableBias()
      (move21): LearnableBias()
      (binary_pw_down1): HardBinaryConv()
      (binary_pw_down2): HardBinaryConv()
      (move22): LearnableBias()
      (prelu2): PReLU(num_parameters=64)
      (move23): LearnableBias()
      (binary_activation): BinaryActivation()
      (pooling): AvgPool2d(kernel_size=2, stride=2, padding=0)
    )
    (2): BasicBlock(
      (move11): LearnableBias()
      (binary_3x3): HardBinaryConv()
      (move12): LearnableBias()
      (prelu1): PReLU(num_parameters=64)
      (move13): LearnableBias()
      (move21): LearnableBias()
      (binary_pw_down1): HardBinaryConv()
      (binary_pw_down2): HardBinaryConv()
  

/opt/conda/envs/ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: [0][  0/167]	Time 16.909 (16.909)	Data  1.319 ( 1.319)	Loss 1.9196e+01 (1.9196e+01)	Acc@1   0.39 (  0.39)	Acc@5   6.25 (  6.25)
Epoch: [0][ 50/167]	Time  0.236 ( 0.529)	Data  0.017 ( 0.046)	Loss 4.7309e+00 (5.2994e+00)	Acc@1   2.73 (  1.05)	Acc@5   6.25 (  5.21)
Epoch: [0][100/167]	Time  0.316 ( 0.359)	Data  0.003 ( 0.029)	Loss 4.6184e+00 (5.0228e+00)	Acc@1   2.73 (  1.45)	Acc@5  12.89 (  6.61)
Epoch: [0][150/167]	Time  0.187 ( 0.311)	Data  0.015 ( 0.024)	Loss 4.4383e+00 (4.8585e+00)	Acc@1   1.56 (  1.95)	Acc@5  14.84 (  8.53)
Test: [ 0/30]	Time  0.965 ( 0.965)	Loss 4.5577e+00 (4.5577e+00)	Acc@1   3.91 (  3.91)	Acc@5  12.50 ( 12.50)
 * acc@1 4.200 acc@5 15.293
learning_rate: 0.0006
Epoch: [1][  0/167]	Time  1.332 ( 1.332)	Data  1.076 ( 1.076)	Loss 4.4081e+00 (4.4081e+00)	Acc@1   5.08 (  5.08)	Acc@5  15.62 ( 15.62)
Epoch: [1][ 50/167]	Time  0.279 ( 0.214)	Data  0.026 ( 0.038)	Loss 4.2289e+00 (4.3032e+00)	Acc@1   6.25 (  4.90)	Acc@5  22.27 ( 18.16)
Epoch: [1][100/167]	Time  0.101 

# Evaluate binarized model on test set

In [11]:
chk=torch.load('./saved_weights/model_best.pth.tar')

In [12]:
chk['best_top1_acc']

tensor(14.1867, device='cuda:0')